In [1]:
import os
cwd = os.getcwd()
import gc
import numpy as np
from dask import dataframe as dd

In [2]:
def rmse(y,h):
    """RMSE
    Args:
        y: real y
        h: predicted y
    Returns:
        RMSE
    """
    a = y-h

    return np.sqrt(sum(a**2)/len(a))

In [3]:
bayes = dd.read_csv('all_predictions_of_naive_bayes.csv')
print("Length of file: %d"%len(bayes))
bayes

Length of file: 120000000


,newbookid,newuser_id,predicted_rating
npartitions=54,,,
,int64,int64,float64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [4]:
os.chdir(cwd+'/../../../data/output csv files')
test = dd.read_csv('test.csv')
os.chdir(cwd)

In [5]:
bayesrank = test.merge(bayes,on = ['newbookid', 'newuser_id'])
bayesrank = bayesrank.map_partitions(lambda x: x.sort_values(by=['newuser_id', 'predicted_rating'], ascending=[1,0]))

In [6]:
print('(1) Bayes Model RMSE: ', np.round(rmse(bayesrank['predicted_rating'],bayesrank['rating']), decimals=3))

(1) Bayes Model RMSE:  1.375


In [7]:
del test, bayesrank
gc.collect()

20

In [8]:
os.chdir(cwd+'/../../../data/output csv files')
train = dd.read_csv('train.csv')
os.chdir(cwd)

In [9]:
bayesranktrain = train.merge(bayes,on = ['newbookid', 'newuser_id'])
bayesranktrain = bayesranktrain.map_partitions(lambda x: x.sort_values(by=['newuser_id', 'predicted_rating'],ascending=[1,0]))

In [10]:
print('(1) Bayes Model Train RMSE: ', np.round(rmse(bayesranktrain['predicted_rating'],bayesranktrain['rating']), decimals=3))

(1) Bayes Model Train RMSE:  1.415


In [11]:
del train, bayesranktrain, bayes
gc.collect()

20